In [1]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage import feature
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
#To return LBP histogram gradient
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
    # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

In [3]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [4]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,9,19,29,39,49,59,69,79,89,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female  male
age                   
(9, 19]     3870  4350
(19, 29]    5760  7590
(29, 39]    1530  2580
(39, 49]    2070  1890
(49, 59]    1590  1170
(59, 69]     420   450
(69, 79]     360    30


In [5]:
def label_fetch(age):
    if age<10:
        return ("0-9")
    elif age<20:
        return ("10-19")
    elif age<30:
        return ("20-29")
    elif age<40:
        return ("30-39")
    elif age<50:
        return ("40-49")
    elif age<60:
        return ("50-59")
    elif age<70:
        return ("60-69")
    elif age<80:
        return ("70-79")
    elif age<90:
        return ("80-89")
    elif age<100:
        return ("90-99")

In [6]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
male_age_label = []
female_age_label = []
male_lbp_features = []
female_lbp_features = []
# initialize the local binary patterns descriptor along with
desc = LocalBinaryPatterns(24, 8)

with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    print(len(image_file_name))
    
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #convert the images to gray and describe it to extract lbp features
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        hist = desc.describe(gray)

        #Set age range label after gender bias
        #appending the lbp features for image onto the lbp data lists 
        if gender[i]== 'male':
            male_lbp_features.append(hist)
            male_age_label.append(label_fetch(age[i]))
        else:
            female_lbp_features.append(hist)
            female_age_label.append(label_fetch(age[i]))
            
    print(len(male_lbp_features))
    print(len(female_lbp_features))

33660
18060
15600


In [7]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(male_age_label)
X =  np.array(male_lbp_features)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

#Linear SVM
print("MALE")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

MALE
Overall Accuracy using Linear SVM: 0.4176079734219269
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      2167
       20-29       0.42      1.00      0.59      3771
       30-39       0.00      0.00      0.00      1316
       40-49       0.00      0.00      0.00       947
       50-59       0.00      0.00      0.00       579
       60-69       0.00      0.00      0.00       235
       70-79       0.00      0.00      0.00        15

    accuracy                           0.42      9030
   macro avg       0.06      0.14      0.08      9030
weighted avg       0.17      0.42      0.25      9030

Accuracy for each class
[0. 1. 0. 0. 0. 0. 0.]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Non-Linear RBF Accuracy: 0.4176079734219269
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      2167
       20-29       0.42      1.00      0.59      3771
       30-39       0.00      0.00      0.00      1316
       40-49       0.00      0.00      0.00       947
       50-59       0.00      0.00      0.00       579
       60-69       0.00      0.00      0.00       235
       70-79       0.00      0.00      0.00        15

    accuracy                           0.42      9030
   macro avg       0.06      0.14      0.08      9030
weighted avg       0.17      0.42      0.25      9030

              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      2167
       20-29       0.42      1.00      0.59      3771
       30-39       0.00      0.00      0.00      1316
       40-49       0.00      0.00      0.00       947
       50-59       0.00      0.00      0.00       579
       60-69       0.00      0.00 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.4176079734219269
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      2167
       20-29       0.42      1.00      0.59      3771
       30-39       0.00      0.00      0.00      1316
       40-49       0.00      0.00      0.00       947
       50-59       0.00      0.00      0.00       579
       60-69       0.00      0.00      0.00       235
       70-79       0.00      0.00      0.00        15

    accuracy                           0.42      9030
   macro avg       0.06      0.14      0.08      9030
weighted avg       0.17      0.42      0.25      9030

              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      2167
       20-29       0.42      1.00      0.59      3771
       30-39       0.00      0.00      0.00      1316
       40-49       0.00      0.00      0.00       947
       50-59       0.00      0.00      0.00       579
       60-69       0.00      0.00

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Iteration 1, loss = 1.78887248
Iteration 2, loss = 1.66344024
Iteration 3, loss = 1.59246276
Iteration 4, loss = 1.55836342
Iteration 5, loss = 1.54091017
Iteration 6, loss = 1.53025297
Iteration 7, loss = 1.52311484
Iteration 8, loss = 1.51808107
Iteration 9, loss = 1.51427143
Iteration 10, loss = 1.51150244
Iteration 11, loss = 1.50922189
Iteration 12, loss = 1.50757364
Iteration 13, loss = 1.50623185
Iteration 14, loss = 1.50509540
Iteration 15, loss = 1.50430993
Iteration 16, loss = 1.50360126
Iteration 17, loss = 1.50291506
Iteration 18, loss = 1.50237635
Iteration 19, loss = 1.50193563
Iteration 20, loss = 1.50150299
Iteration 21, loss = 1.50111840
Iteration 22, loss = 1.50079780
Iteration 23, loss = 1.50050686
Iteration 24, loss = 1.50011896
Iteration 25, loss = 1.49993811
Iteration 26, loss = 1.49968522
Iteration 27, loss = 1.49949348
Iteration 28, loss = 1.49931581
Iteration 29, loss = 1.49910442
Iteration 30, loss = 1.49906457
Iteration 31, loss = 1.49884128
Iteration 32, los

Iteration 254, loss = 1.48079069
Iteration 255, loss = 1.48073787
Iteration 256, loss = 1.48061092
Iteration 257, loss = 1.48048176
Iteration 258, loss = 1.48045817
Iteration 259, loss = 1.48040913
Iteration 260, loss = 1.48007935
Iteration 261, loss = 1.48007089
Iteration 262, loss = 1.48007688
Iteration 263, loss = 1.47990543
Iteration 264, loss = 1.47972207
Iteration 265, loss = 1.47967803
Iteration 266, loss = 1.47950591
Iteration 267, loss = 1.47950575
Iteration 268, loss = 1.47932682
Iteration 269, loss = 1.47929351
Iteration 270, loss = 1.47929671
Iteration 271, loss = 1.47924290
Iteration 272, loss = 1.47893084
Iteration 273, loss = 1.47896457
Iteration 274, loss = 1.47885502
Iteration 275, loss = 1.47878788
Iteration 276, loss = 1.47866306
Iteration 277, loss = 1.47873535
Iteration 278, loss = 1.47848471
Iteration 279, loss = 1.47854458
Iteration 280, loss = 1.47837671
Iteration 281, loss = 1.47820550
Iteration 282, loss = 1.47806861
Iteration 283, loss = 1.47798430
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-

In [8]:
# Fit a SVM classifier to the data where gender=female
y =  np.array(female_age_label)
X = np.array(female_lbp_features)   
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("FEMALE")
print("Overall Accuracy using Linear SVM:"+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0
    
#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
    
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')   

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
    
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
    
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')


FEMALE
Overall Accuracy using Linear SVM:0.37038461538461537
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      1937
       20-29       0.37      1.00      0.54      2889
       30-39       0.00      0.00      0.00       774
       40-49       0.00      0.00      0.00      1032
       50-59       0.00      0.00      0.00       778
       60-69       0.00      0.00      0.00       220
       70-79       0.00      0.00      0.00       170

    accuracy                           0.37      7800
   macro avg       0.05      0.14      0.08      7800
weighted avg       0.14      0.37      0.20      7800

Accuracy for each class
[0. 1. 0. 0. 0. 0. 0.]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      1937
       20-29       0.37      1.00      0.54      2889
       30-39       0.00      0.00      0.00       774
       40-49       0.00      0.00      0.00      1032
       50-59       0.00      0.00      0.00       778
       60-69       0.00      0.00      0.00       220
       70-79       0.00      0.00      0.00       170

    accuracy                           0.37      7800
   macro avg       0.05      0.14      0.08      7800
weighted avg       0.14      0.37      0.20      7800

Accuracy for each class
[0. 1. 0. 0. 0. 0. 0.]




Non-Linear RBF Accuracy: 0.37038461538461537
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      1937
       20-29       0.37      1.00      0.54      2889
       30-39       0.00      0.00      0.00       774
       40-49       0.00      0.00      0.00      1032
       50-59       0.00      0.00   

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Iteration 3, loss = 1.70274973
Iteration 4, loss = 1.67378342
Iteration 5, loss = 1.65764588
Iteration 6, loss = 1.64820823
Iteration 7, loss = 1.64248959
Iteration 8, loss = 1.63875950
Iteration 9, loss = 1.63616306
Iteration 10, loss = 1.63430415
Iteration 11, loss = 1.63294751
Iteration 12, loss = 1.63196850
Iteration 13, loss = 1.63124130
Iteration 14, loss = 1.63070434
Iteration 15, loss = 1.63035705
Iteration 16, loss = 1.62996465
Iteration 17, loss = 1.62971627
Iteration 18, loss = 1.62954653
Iteration 19, loss = 1.62936904
Iteration 20, loss = 1.62923373
Iteration 21, loss = 1.62914051
Iteration 22, loss = 1.62907981
Iteration 23, loss = 1.62896606
Iteration 24, loss = 1.62901169
Iteration 25, loss = 1.62887768
Iteration 26, loss = 1.62880026
Iteration 27, loss = 1.62882149
Iteration 28, loss = 1.62872044
Iteration 29, loss = 1.62869530
Iteration 30, loss = 1.62869530
Iteration 31, loss = 1.62863357
Iteration 32, loss = 1.62868865
Iteration 33, loss = 1.62854759
Iteration 34, l

Iteration 257, loss = 1.61699317
Iteration 258, loss = 1.61690097
Iteration 259, loss = 1.61682979
Iteration 260, loss = 1.61675858
Iteration 261, loss = 1.61671626
Iteration 262, loss = 1.61659724
Iteration 263, loss = 1.61654349
Iteration 264, loss = 1.61646685
Iteration 265, loss = 1.61635295
Iteration 266, loss = 1.61628327
Iteration 267, loss = 1.61621615
Iteration 268, loss = 1.61615576
Iteration 269, loss = 1.61613904
Iteration 270, loss = 1.61611655
Iteration 271, loss = 1.61595177
Iteration 272, loss = 1.61583670
Iteration 273, loss = 1.61575340
Iteration 274, loss = 1.61576936
Iteration 275, loss = 1.61562893
Iteration 276, loss = 1.61556650
Iteration 277, loss = 1.61557042
Iteration 278, loss = 1.61545084
Iteration 279, loss = 1.61540440
Iteration 280, loss = 1.61537465
Iteration 281, loss = 1.61529530
Iteration 282, loss = 1.61521773
Iteration 283, loss = 1.61509058
Iteration 284, loss = 1.61504853
Iteration 285, loss = 1.61500455
Iteration 286, loss = 1.61491801
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-